In [ ]:
import numpy as np
import scipy.spatial
import scipy.stats
import pandas as pd
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold

In [ ]:
path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Movielens/user_movie.dat'

In [ ]:
names = ['user_id', 'item_id', 'rating', 'time']
df = pd.read_csv(path, sep='\t', names=names).iloc[:, :3]

'''filtering'''
grouped_df = df.groupby(['user_id'])['item_id'].count()
filtered_user_id = grouped_df[grouped_df>3].index
filtered_df = df.set_index('user_id').loc[filtered_user_id].reset_index() 

n_user= filtered_df['user_id'].max()
n_item = filtered_df['item_id'].max()
data= np.array(filtered_df)
print(data.shape)
print(f'n_user: {n_user}')

(100000, 3)
n_user: 943


In [ ]:
matrix = np.zeros(shape= (n_user, n_item), dtype=float)
for info in data:
  matrix[info[0]-1, info[1]-1] = info[2]

# Function

In [ ]:
def similarity(matrix, metric= 'cosine'):
  mat = np.zeros(shape= (n_user, n_user), dtype= float)
  for i_m, row_m in enumerate(matrix[:n_user]):
    # print('ok')
    for i_s, row_s in enumerate(matrix[i_m+1:]):
      if metric == 'cosine':
        mat[i_m][i_s] = 1-scipy.spatial.distance.cosine(matrix[i_m],matrix[i_s])
      elif metric == 'spearson':
        mat[i_m][i_s] = scipy.stats.pearsonr(matrix[i_m],matrix[i_s])[0]
  return mat 

In [ ]:
similarity_matrix = similarity(matrix= matrix, metric= 'cosine')

# Training

In [ ]:
k= 30

In [ ]:
# test_kf = KFold(n_splits=5, shuffle= True, random_state=42)
# val_kf = KFold(n_splits=8, shuffle= True, random_state=42)
kf= KFold(n_splits=5, shuffle= True, random_state=42)

In [ ]:
RMSEs= []
for train_indice, test_indice in kf.split(data):
  print('start round...')
  train_data= data[train_indice]
  y= []
  pred= []
  train_user_ids = set([info[0]-1 for info in train_data])
  id_simi_dict = dict()
  # u = np.mean([info[2] for info in train_data]) # overall mean item rating

  '''iterate over each user in test'''
  for (test_user_id, test_item_id, rating) in data[test_indice]:
    # bu = np.mean([info[2] for info in train_data if info[0]==test_user_id])
    # bi = np.mean([info[2] for info in train_data if info[1]==test_item_id])

    # top_k_matched_users = similarity(test_user_id, train_matrix)
    for train_user_id in train_user_ids: 
      id_simi_dict[train_user_id] = similarity_matrix[test_user_id-1, train_user_id-1] # 該user 和train_users相似度

    top_k_users = sorted(id_simi_dict.items(),key= lambda x: x[1] ,reverse=True)[:k] # list of tuples of (train_user_id, similarity)
    simi_r_sum = sum([matrix[info[0]-1, test_item_id-1]*info[1] for info in top_k_users])
    simi_sum = sum([info[1] for info in top_k_users])
    # print(simi_r_sum/simi_sum)
    if simi_sum != 0:
      pred.append(simi_r_sum/simi_sum)
    else: 
      pred.append(0)
    y.append(rating)

  rmse = mean_squared_error(y, pred, squared=False)
  RMSEs.append(rmse)
print(f'avg RMSE: {np.mean(RMSEs)}')

start round...
start round...
start round...
start round...
start round...
avg RMSE: 2.127987169704867


In [ ]:
np.mean(RMSEs)

2.127987169704867

In [ ]:
"""
Cosine: avg rse: 2.1
Spearson：avg rse: 2.25
"""

'\nCosine: avg rse: 2.1\nSpearson：avg rse: 2.25\n'